---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open('university_towns.txt', 'r') as file:
        tod = file.readlines()
    dic = {'State': [], 'RegionName': []}
    for line in tod:
        if '[edit]' in line:
            stt = line[:line.index('[')]
            continue
        else:
            if '(' in line:
                line = line[:line.index('(')-1]
            else:
                line = line.strip()
            dic['State'].append(stt)
            dic['RegionName'].append(line)
    df = pd.DataFrame(dic, columns=['State', 'RegionName'])    
    return df
# get_list_of_university_towns()#.groupby('State').size()
# import re
# import pandas as pd
# import numpy as np
# # list of unique states
# stateStr = """
# Ohio, Kentucky, American Samoa, Nevada, Wyoming
# ,National, Alabama, Maryland, Alaska, Utah
# ,Oregon, Montana, Illinois, Tennessee, District of Columbia
# ,Vermont, Idaho, Arkansas, Maine, Washington
# ,Hawaii, Wisconsin, Michigan, Indiana, New Jersey
# ,Arizona, Guam, Mississippi, Puerto Rico, North Carolina
# ,Texas, South Dakota, Northern Mariana Islands, Iowa, Missouri
# ,Connecticut, West Virginia, South Carolina, Louisiana, Kansas
# ,New York, Nebraska, Oklahoma, Florida, California
# ,Colorado, Pennsylvania, Delaware, New Mexico, Rhode Island
# ,Minnesota, Virgin Islands, New Hampshire, Massachusetts, Georgia
# ,North Dakota, Virginia
# """
# #list of regionName entries string length
# regNmLenStr = """
# 06,08,12,10,10,04,10,08,09,09,05,06,11,06,12,09,08,10,12,06,
# 06,06,08,05,09,06,05,06,10,28,06,06,09,06,08,09,10,35,09,15,
# 13,10,07,21,08,07,07,07,12,06,14,07,08,16,09,10,11,09,10,06,
# 11,05,06,09,10,12,06,06,11,07,08,13,07,11,05,06,06,07,10,08,
# 11,08,13,12,06,04,08,10,08,07,12,05,06,09,07,10,16,10,06,12,
# 08,07,06,06,06,11,14,11,07,06,06,12,08,10,11,06,10,14,04,11,
# 18,07,07,08,09,06,13,11,12,10,07,12,07,04,08,09,09,13,08,10,
# 16,09,10,08,06,08,12,07,11,09,07,09,06,12,06,09,07,10,09,10,
# 09,06,15,05,10,09,11,12,10,10,09,13,06,09,11,06,11,09,13,37,
# 06,13,06,09,49,07,11,12,09,11,11,07,12,10,06,06,09,04,09,15,
# 10,12,05,09,08,09,09,07,14,06,07,16,12,09,07,09,06,32,07,08,
# 08,06,10,36,09,10,09,06,09,11,09,06,10,07,14,08,07,06,10,09,
# 05,11,07,06,08,07,05,07,07,04,06,05,09,04,25,06,07,08,05,08,
# 06,05,11,09,07,07,06,13,09,05,16,05,10,09,08,11,06,06,06,10,
# 09,07,06,07,10,05,08,07,06,08,06,30,09,07,06,11,07,12,08,09,
# 16,12,11,08,06,04,10,10,15,05,11,11,09,08,06,04,10,10,07,09,
# 11,08,26,07,13,05,11,03,08,07,06,05,08,13,10,08,08,08,07,07,
# 09,05,04,11,11,07,06,10,11,03,04,06,06,08,08,06,10,09,05,11,
# 07,09,06,12,13,09,10,11,08,07,07,08,09,10,08,10,08,56,07,12,
# 07,16,08,04,10,10,10,10,07,09,08,09,09,10,07,09,09,09,12,14,
# 10,29,19,07,11,12,13,13,09,10,12,12,12,08,10,07,10,07,07,08,
# 08,08,09,10,09,11,09,07,09,10,11,11,10,09,09,12,09,06,08,07,
# 12,09,07,07,06,06,08,06,15,08,06,06,10,10,10,07,05,10,07,11,
# 09,12,10,12,04,10,05,05,04,14,07,10,09,07,11,10,10,10,11,15,
# 09,14,12,09,09,07,12,04,10,10,06,10,07,28,06,10,08,09,10,10,
# 10,13,12,08,10,09,09,07,09,09,07,11,11,13,08,10,07
# """

# df = get_list_of_university_towns()

# cols = ["State", "RegionName"]

# print('Shape test: ', "Passed" if df.shape ==
#       (517, 2) else 'Failed')
# print('Index test: ',
#       "Passed" if df.index.tolist() == list(range(517))
#       else 'Failed')

# print('Column test: ',
#       "Passed" if df.columns.tolist() == cols else 'Failed')
# print('\\n test: ',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\n')) or any(df[cols[1]].str.contains('\n'))
#       else 'Passed')
# print('Trailing whitespace test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\s+$')) or any(df[cols[1]].str.contains(
#               '\s+$'))
#       else 'Passed')
# print('"(" test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\(')) or any(df[cols[1]].str.contains(
#               '\('))
#       else 'Passed')
# print('"[" test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\[')) or any(df[cols[1]].str.contains(
#               '\]'))
#       else 'Passed')

# states_vlist = [st.strip() for st in stateStr.split(',')]

# mismatchedStates = df[~df['State'].isin(
#     states_vlist)].loc[:, 'State'].unique()
# print('State test: ', "Passed" if len(
#     mismatchedStates) == 0 else "Failed")
# if len(mismatchedStates) > 0:
#     print()
#     print('The following states failed the equality test:')
#     print()
#     print('\n'.join(mismatchedStates))

# df['expected_length'] = [int(s.strip())
#                          for s in regNmLenStr.split(',')
#                          if s.strip().isdigit()]
# regDiff = df[df['RegionName'].str.len() != df['expected_length']].loc[
#     :, ['RegionName', 'expected_length']]
# regDiff['actual_length'] = regDiff['RegionName'].str.len()
# print('RegionName test: ', "Passed" if len(regDiff) ==
#       0 else ' \nMismatching regionNames\n {}'.format(regDiff))

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=5)[['Unnamed: 4', 'GDP in billions of chained 2009 dollars.1']][2:]
    df = df.rename(columns={'Unnamed: 4': 'Quarters', 'GDP in billions of chained 2009 dollars.1': 'GDP(billions)'})
    df = df.set_index('Quarters')
    df = df.drop(df.loc[:'1999q4'].index, axis=0)
    for i in range(2, len(df.index)-2):
        cond = df.iloc[i-2][0] > df.iloc[i-1][0] and df.iloc[i-1][0] > df.iloc[i][0] and df.iloc[i][0] < df.iloc[i+1][0] and df.iloc[i+1][0] < df.iloc[i+2][0]
        if cond:
            j = i-1
            while True:
                if df.iloc[j][0] < df.iloc[j+1][0]:
                    return df.iloc[j+2].name
                j -= 1
# get_recession_start()

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=5)[['Unnamed: 4', 'GDP in billions of chained 2009 dollars.1']][2:]
    df = df.rename(columns={'Unnamed: 4': 'Quarters', 'GDP in billions of chained 2009 dollars.1': 'GDP(billions)'})
    df = df.set_index('Quarters')
    df = df.drop(df.loc[:'1999q4'].index, axis=0)
    for i in range(2, len(df.index)-2):
        cond = df.iloc[i-2][0] > df.iloc[i-1][0] and df.iloc[i-1][0] > df.iloc[i][0] and df.iloc[i][0] < df.iloc[i+1][0] and df.iloc[i+1][0] < df.iloc[i+2][0]
        if cond:
            return df.iloc[i+2].name
# get_recession_end()

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=5)[['Unnamed: 4', 'GDP in billions of chained 2009 dollars.1']][2:]
    df = df.rename(columns={'Unnamed: 4': 'Quarters', 'GDP in billions of chained 2009 dollars.1': 'GDP(billions)'})
    df = df.set_index('Quarters')
    df = df.drop(df.loc[:'1999q4'].index, axis=0)
    for i in range(2, len(df.index)-2):
        cond = df.iloc[i-2][0] > df.iloc[i-1][0] and df.iloc[i-1][0] > df.iloc[i][0] and df.iloc[i][0] < df.iloc[i+1][0] and df.iloc[i+1][0] < df.iloc[i+2][0]
        if cond:
            return df.iloc[i].name
# get_recession_bottom()

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df = df.set_index(['State', 'RegionName'])
    df = df.rename(index=states)
    to_drop = ['199{}-{:02}'.format(n, m) for n in range(6, 10) for m in range(1, 13)][3:]
    df = df.drop(to_drop+['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1)
    for y in range(17):
        for q in range(1, 5):
            if not (y == 16 and q >= 3):
                df['20{:02}q{}'.format(y, q)] = df[['20{:02}-{:02}'.format(y, m) for m in range(-2+3*q, 3*q+1)]].mean(axis=1)
            elif y == 16 and q == 3:
                df['20{:02}q{}'.format(y, q)] = df[['20{:02}-{:02}'.format(y, m) for m in range(-2+3*q, 3*q)]].mean(axis=1)
    to_drop = ['20{:02}-{:02}'.format(n, m) for n in range(17) for m in range(1, 13)][:-4]
    df = df.drop(to_drop, axis=1)
    
    
    return df
# convert_housing_data_to_quarters().loc[[('Ohio','Akron'),('Ohio','Dayton')]].loc[:,['2010q3','2015q2','2016q4']]

In [12]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    univ = get_list_of_university_towns()['RegionName']
    start = get_recession_start()
    before = start[:-1] + str(int(start[-1])-1) if not start[-1] == '1' else '20{:02}q4'.format(int(start[2:4])-1)
    bottom = get_recession_bottom()
    house = convert_housing_data_to_quarters().loc[:, before:bottom].dropna()
    house_us = house.drop(univ.values, level='RegionName', axis=0)
    house_univ = house.drop(house_us.index.get_level_values(level='RegionName'), level='RegionName', axis=0)
    del house, univ
    house_us['PriceRatio'] = house_us[before]/house_us[bottom]
    house_univ['PriceRatio'] = house_univ[before]/house_univ[bottom]
#     print(house_univ)
#     print(house_us)
    test = ttest_ind(house_us['PriceRatio'], house_univ['PriceRatio'])
#     print(test)
    different = True if test.pvalue < 0.01 else False
    better = 'university town' if house_univ['PriceRatio'].mean() < house_us['PriceRatio'].mean() else 'non-university town'
    
    return (different, test.pvalue, better)
# run_ttest()

                                    2008q2         2008q3         2008q4  \
State          RegionName                                                  
Nevada         Las Vegas     232300.000000  213366.666667  194933.333333   
California     San Diego     441400.000000  424666.666667  407633.333333   
Texas          Dallas        115366.666667  112166.666667  109900.000000   
Florida        Jacksonville  160433.333333  154733.333333  149166.666667   
Texas          Austin        213733.333333  211033.333333  207466.666667   
Ohio           Columbus      113500.000000  112200.000000  111600.000000   
Tennessee      Memphis        84400.000000   83700.000000   80900.000000   
Massachusetts  Boston        347966.666667  347333.333333  341133.333333   
Tennessee      Nashville     154600.000000  154333.333333  154200.000000   
Wisconsin      Milwaukee     131233.333333  127900.000000  124500.000000   
Arizona        Tucson        180600.000000  175766.666667  172133.333333   
California  

(True, 6.7550789357461301e-05, 'university town')